# Multilayer Output

In earlier deployed ML production systems, models were treated as independent algorithms, and we would be interested only in the final output (the prediction). Today,
we build not models, but applications that are an amalgamation, or composition, of models. As a result, we no longer treat the task component as a single output.

Instead, we see it as having four outputs, depending on how the model is connected to other models in the application. These outputs are as follows:
1. Feature Extraction:
– High dimensionality (encoding)
– Low dimensionality (embedding)—feature vector
2. Prediction:
– Prediction pre-activation (probabilities)—soft target
– Post-activation (outputs)—hard targets

In the multi-output of a conventional classifier depicted in figure 5.17, you can see that the input to the task component is also an independent output of the model, referred to as the encoding. The encoding is then passed through a global average pooling for dimensionality reduction, further reducing the size of the features extracted by the learner component. The output from the global average pooling is also an independent output of the model, referred to as the embedding.

<img src="img_1.png" />

The embedding is then passed to a pre-activation dense layer (prior to the softmax activation). The output from the pre-activation layer is also an independent output of
the model, referred to as the pre-activation probability distribution. This probability distribution is then passed through a softmax for the post-activation probability distribution, making the fourth independent output of the model. All these outputs can then be used by downstream tasks.

Let’s describe a simple real-world example for using a multi-output task component: estimating the cost of repairs from a photo of a vehicle. We want estimates on two categories:
1. costs for minor damage like dings and scratches, and
2. costs for major damage like collision damage.
We could attempt to do this in a single task component that acts as a regressor to output a real value (dollar value), but we would really be overloading the task component during training because it’s learning both tiny values (minor damage) and large values (major damage). During training, the wide distribution in the values likely will keep the model from converging.

 The approach is to solve this as two separate task components:
 1. one for minor - learn only tiny values - and
 2. one for major damage -learn only large values-.
The minor damage task component will learn only tiny values, and the major damage task component will learn only large values—so both task components should converge during training.


Next, we consider which output level we share with each of the two tasks.
1. For the minor damage, we are looking at tiny objects. While we are not covering object detection, the historical problem with object classification with small objects was that the cropped feature maps after being pooled contained too little spatial information. The fix was to do the object classification from feature maps at an earlier convolution; the feature maps would then be of sufficient size, so that when a tiny object is cropped out, enough spatial information remains for object classification.
2. We have a comparable issue in our example. For minor damage, the objects (each ding) will be very small, and we need larger feature maps to detect them. So for this
purpose, we connect the high-dimensional encoding, prior to averaging and pooling, to the task that performs minor damage estimating. On the other hand, major collision damage does not require much detail. If the fender has a dent, it has to be replaced no matter the size or location of the dent, for instance. So for this purpose,
we connect the low-dimensional embedding, after averaging and pooling, to the task that performs major damage estimating. Figure 5.18 illustrates this example.

<img src="img_2.png" />

The following is an example implementation of coding the multi-outputs to a classifier component. The feature extraction and prediction outputs are implemented by capturing the tensor inputs to each layer. At the end of the classifier, we replace returning a single output with returning a tuple of all four outputs:


<img src="img_3.png"/>

In [1]:
from  keras.layers import GlobalAveragePooling2D, Dense, Activation
def classifier(inputs, n_classes):
    """

    :param inputs:
    :param n_classes:
    :return: encodeing,  embedding, probabilities, outputs
    """
    encoding = inputs
    embedding = GlobalAveragePooling2D()(inputs)
    probabilities = Dense(n_classes)(embedding)
    outputs = Activation("softmax")(probabilities)
    return encoding, embedding, probabilities, outputs